1\. **Text files**

Perform the following operations on plain `txt` files:

+ create a list of integrer numbers and then save it to a text file named `data_int.txt`. Run the `cat` command to print the content of the file.
+ create a matrix of 5x5 floats and then save it to a text file named `data_float.txt`. Use the `cat` command to print the content of the file.
+ load the `txt` file of the previous point and convert it to a `csv` file by hand.

In [ ]:
import pandas as pd
import numpy as np

data_int = [1,2,3,4,5,6,7,8,9]

i = open("data_int.txt", "w")
i.write(str(data_int))
i.close()

!type ".\data_int.txt"


data_float = np.random.random((5,5))
f = open("data_float.txt", "w")
f.write(str(data_float))
f.close

!type ".\data_float.txt"

read_int_file = pd.read_csv (r"D:\Desktop\FirstSemester\SCwP Exercise\CopyOnes\06ex\data_int.txt")
read_int_file.to_csv (r"D:\Desktop\FirstSemester\SCwP Exercise\CopyOnes\06ex\data_int.csv", index=None)

read_float_file = pd.read_csv (r"D:\Desktop\FirstSemester\SCwP Exercise\CopyOnes\06ex\data_float.txt")
read_float_file.to_csv (r"D:\Desktop\FirstSemester\SCwP Exercise\CopyOnes\06ex\data_float.txt", index=None)




2\. **JSON files**

Load the file *user_data.json*, which can be found at:

- https://www.dropbox.com/s/sz5klcdpckc39hd/user_data.json

and filter the data by the "CreditCardType" when it equals to "American Express". Than save the data to a new CSV file.

In [ ]:
#url = "https://www.dropbox.com/s/sz5klcdpckc39hd/user_data.json"
#data = pd.read_json(url) # use sep="," for coma separation. 

data = pd.read_json("user_data.json")
american = data[data["CreditCardType"] == "American Express"]
pd.DataFrame(american).to_csv('American Express.csv') 
display(american)



3\. **CSV files with Pandas**

Load the file from this url:

- https://www.dropbox.com/s/kgshemfgk22iy79/mushrooms_categorized.csv

with Pandas. 

+ explore and print the DataFrame
+ calculate, using `groupby()`, the average value of each feature, separately for each class
+ save the file in a JSON format.

In [ ]:
import pandas as pd
import numpy as np
import csv

data = pd.read_csv("mushrooms_categorized.csv")

def average(input):
    return data.groupby([input]).mean()

#print("Class Name:",i,"\n")
#print(average("class"))
display(average("class"))
pd.DataFrame(average("class")).to_json('mushroomns_categorized.json')  



4\. **Reading the credit card numbers**

Get the binary file named *credit_card.dat* from this address:

- https://www.dropbox.com/s/8m0syw2tkul3dap/credit_card.dat

and convert the data into the real credit card number, knowing that:
- each line corresponds to a credit card number, which consists of 16 characters (which are numbers in the 0-9 range) divided in 4 blocks, with a whitespace between each block
- each character is written using a 6 bit binary representation (including the whitespace)
- the final 4 bits of each line are a padding used to determine the end of the line, and can be ignored

*Hint*: convert the binary numbers to the decimal representation first, and then use the `chr()` function to convert the latter to a char

In [ ]:
import pandas as pd
import numpy as np
import csv  

data = open("credit_card.dat", "r").readlines()

credit_cards = []
for a in data:
    n = 6    # 6 bit binary representation
    bin = [a[i:i+n] for i in range(0, len(a)-5, n)]  # -5 because of \n 
    number = ""
    for char in bin:
        number += chr(int(char,2))

    credit_cards.append(number)
credit_cards



5\. **Write data to a binary file**

a) Start from the `data/data_000637.txt` file that we have used during the previous lectures, and convert it to a binary file according to the format defined below:

In [34]:
from IPython.display import Image
Image("images/data_format.png")

import pandas as pd
import numpy as np
import csv  
import struct
import os

file_name = "data_000637.txt" 
file_dat = "data_000637.dat"
A=10
df_c = pd.read_csv(file_name, nrows=10)


with open('data_000637_binary.dat', 'wb') as binary_file:    
    
    for index, row in df_c.iterrows():
        word = 0
        word = (row['HEAD'] << 62) | word
        word = (row['FPGA'] << 58) | word
        word = (row['TDC_CHANNEL'] << 49) | word
        word = (row['ORBIT_CNT'] << 17) | word
        word = (row['BX_COUNTER'] << 5 ) | word
        word = (row['TDC_MEAS'] << 0 ) | word
        binary_file.write(struct.pack("<q", word))
        
data = {}
columns = ['HEAD', 'FPGA', 'CHANNEL', 'ORBIT_CNT', 'BX_CNT', 'TDC_MEAS']
df = pd.DataFrame({}, columns=columns)

with open('data_000637_binary.dat', 'rb') as file:
    file_content = file.read()
    cnt = 0
    word_size = 8
    
    for i in range(0, len(file_content), word_size):
        
        cnt= cnt+1
        if cnt > 10: break
        word = struct.unpack("<q", file_content[i : i + word_size])[0] 
        head     = (word >> 62) & 0x3
        fpga     = (word >> 58) & 0xF
        tdc_chan = (word >> 49) & 0x1FF
        orb_cnt  = (word >> 17) & 0xFFFFFFFF
        bx       = (word >> 5 ) & 0xFFF
        tdc_meas = (word >> 0 ) & 0x1F
        entry = {'HEAD' : head, 'FPGA' : fpga, 'CHANNEL' : tdc_chan, 'ORBIT_CNT' : orb_cnt, 'BX_CNT' : bx, 'TDC_MEAS' : tdc_meas}
        df = df.append(entry, ignore_index=True)
        if cnt > 10: 
            break
      
print(df)
print(df_c)

df1 = pd.read_csv(file_name, nrows = 10)
df1.to_csv(r"./data_000637_new.txt")
new="./data_000637_new.txt"
        
data_000637_txt = os.stat(new)
data_000637_binary_dat = os.stat(file_dat)

print("\nThe .txt file's size: ", (data_000637_txt.st_size)," B\n")
print("The .dat file's size: ", ((data_000637_binary_dat.st_size)/(1024*1024))," MB\n")
print("The difference between the file sizes as MB: ", ((data_000637_txt.st_size-data_000637_binary_dat.st_size)/(1024**2)))



     


  HEAD FPGA CHANNEL   ORBIT_CNT BX_CNT TDC_MEAS
0    1    0     123  3869200167   2374       26
1    1    0     124  3869200167   2374       27
2    1    0      63  3869200167   2553       28
3    1    0      64  3869200167   2558       19
4    1    0      64  3869200167   2760       25
5    1    0      63  3869200167   2762        4
6    1    0      61  3869200167   2772       14
7    1    0     139  3869200167   2776        0
8    1    0      62  3869200167   2774       21
9    1    0      60  3869200167   2788        7
   HEAD  FPGA  TDC_CHANNEL   ORBIT_CNT  BX_COUNTER  TDC_MEAS
0     1     0          123  3869200167        2374        26
1     1     0          124  3869200167        2374        27
2     1     0           63  3869200167        2553        28
3     1     0           64  3869200167        2558        19
4     1     0           64  3869200167        2760        25
5     1     0           63  3869200167        2762         4
6     1     0           61  3869200167       

C:\Users\AnilOzfirat\AppData\Local\Temp\ipykernel_15600\3830809893.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(entry, ignore_index=True)
C:\Users\AnilOzfirat\AppData\Local\Temp\ipykernel_15600\3830809893.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(entry, ignore_index=True)
C:\Users\AnilOzfirat\AppData\Local\Temp\ipykernel_15600\3830809893.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(entry, ignore_index=True)
C:\Users\AnilOzfirat\AppData\Local\Temp\ipykernel_15600\3830809893.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(entry, ignore_index=True)


*Hints*:
- Read the first 10 lines using Pandas
- Iterate over the DataFrame rows
- For every row, ``pack'' the values (features) into a single 64-bit word, according to the format specified above. Use bit-wise shifts and operators to do so.
- Write each 64-bit word to a binary file. You can use `struct` in this way:
```
binary_file.write( struct.pack('<q', word) )
```
where `word` is the 64-bit word.
- Close the file after completing the loop.

b) Check that the binary file is correctly written by reading it with the code used in the lecture `06_OSEMN.ipynb`, and verify that the content of the `txt` and binary files is consistent.

c) What is the difference of the size on disk between equivalent `txt` and binary files?